Wide World Importers Data Analytics with PySpark

DESCRIPTION:

Wide World Importers (WWI) is a goods importer and distributor operating from the San Francisco.

As a wholesaler, WWI's customers are mostly companies who resell to individuals. WWI sells to retail customers across the United States including specialty stores, supermarkets, computing stores, tourist attraction shops, and some individuals. WWI also sells to other wholesalers via a network of agents who promote the products on WWI's behalf. While all of WWI's customers are currently based in the United States, the company is intending to push for expansion into other countries/regions.

WWI buys goods from suppliers including novelty and toy manufacturers, and other novelty wholesalers. They stock the goods in their WWI warehouse and reorder from suppliers as needed to fulfill customer orders. They also purchase large volumes of packaging materials, and sell these in smaller quantities as a convenience for the customers.

SOURCE:

WWI dataset that is used in this project was downloaded from DataCamp source.

IMPORTING LIBRARIES AND STARTING SPARK SESSION:

In [2]:
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

from pyspark.sql.functions import col,isnan,when,count
from pyspark.sql.types import IntegerType

spark = SparkSession.builder.appName("WWI_Analysis").getOrCreate()

KOALAS LIBRARY:

Koalas supports Apache Spark 3.1 and below as it is officially included to PySpark in Apache Spark 3.2 and above.
Spark version 3.3.2 works here, so in this project PySpark will be used directly.

In [4]:
spark.version

'3.3.2'

READING DATA FILES:

FACT INTERNET SALE TABLE

The main table in the WWI tabel hirarhy, contains important information about the actual sales of products. It has 8 keys that link this table with others. All data types are correct.

In [6]:
fact_internet_sale = spark.read.option('header', 'true').csv('Tables_WWI/FactInternetSale.csv', inferSchema=True)
fact_internet_sale.printSchema()

root
 |-- ProductKey: integer (nullable = true)
 |-- OrderDateKey: integer (nullable = true)
 |-- DueDateKey: integer (nullable = true)
 |-- ShipDateKey: integer (nullable = true)
 |-- CustomerKey: integer (nullable = true)
 |-- PromotionKey: integer (nullable = true)
 |-- CurrencyKey: integer (nullable = true)
 |-- SalesTerritoryKey: integer (nullable = true)
 |-- SalesOrderNumber: string (nullable = true)
 |-- SalesOrderLineNumber: integer (nullable = true)
 |-- RevisionNumber: integer (nullable = true)
 |-- OrderQuantity: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- ExtendedAmount: double (nullable = true)
 |-- UnitPriceDiscountPct: integer (nullable = true)
 |-- DiscountAmount: integer (nullable = true)
 |-- ProductStandardCost: double (nullable = true)
 |-- TotalProductCost: double (nullable = true)
 |-- SalesAmount: double (nullable = true)
 |-- TaxAmt: double (nullable = true)
 |-- Freight: double (nullable = true)
 |-- CarrierTrackingNumber: string (n

In [7]:
rows = fact_internet_sale.count()
print(f"fact_internet_sale DataFrame Rows count : {rows}")

fact_internet_sale DataFrame Rows count : 60398


Extract the required data from the Fact Internet Sale table using the spark.sql file, changing some of the names to make them clearer:

In [80]:
fact_internet_sale.createOrReplaceTempView('FactInternetSale')
fact_int_sale_sql = spark.sql('SELECT ProductKey, OrderDateKey, SalesTerritoryKey, OrderQuantity AS UnitsSold, TotalProductCost, SalesAmount AS Revenue, TaxAmt AS Taxes, Freight AS DeliveryCost FROM FactInternetSale')
fact_int_sale_sql.show(5)

+----------+------------+-----------------+---------+----------------+--------+--------+------------+
|ProductKey|OrderDateKey|SalesTerritoryKey|UnitsSold|TotalProductCost| Revenue|   Taxes|DeliveryCost|
+----------+------------+-----------------+---------+----------------+--------+--------+------------+
|       310|    20110701|                6|        1|       2171.2942| 3578.27|286.2616|     89.4568|
|       346|    20110701|                7|        1|       1912.1544| 3399.99|271.9992|     84.9998|
|       346|    20110701|                1|        1|       1912.1544| 3399.99|271.9992|     84.9998|
|       336|    20110701|                4|        1|        413.1463|699.0982| 55.9279|     17.4775|
|       346|    20110701|                9|        1|       1912.1544| 3399.99|271.9992|     84.9998|
+----------+------------+-----------------+---------+----------------+--------+--------+------------+
only showing top 5 rows



Creating "Profit" column: 

In [89]:
profit_sale_sql = fact_int_sale_sql.withColumn("Profit", fact_int_sale_sql.Revenue - (fact_int_sale_sql.TotalProductCost + fact_int_sale_sql.Taxes))
profit_sale_sql.show(3)

+----------+------------+-----------------+---------+----------------+-------+--------+------------+--------------+------------------+
|ProductKey|OrderDateKey|SalesTerritoryKey|UnitsSold|TotalProductCost|Revenue|   Taxes|DeliveryCost|RevenueRevenue|            Profit|
+----------+------------+-----------------+---------+----------------+-------+--------+------------+--------------+------------------+
|       310|    20110701|                6|        1|       2171.2942|3578.27|286.2616|     89.4568|          3578|1120.7142000000003|
|       346|    20110701|                7|        1|       1912.1544|3399.99|271.9992|     84.9998|          3399|1215.8364000000001|
|       346|    20110701|                1|        1|       1912.1544|3399.99|271.9992|     84.9998|          3399|1215.8364000000001|
+----------+------------+-----------------+---------+----------------+-------+--------+------------+--------------+------------------+
only showing top 3 rows



DATE TABLE

Contains day, month and year data linked to the FactSale table, which explains the time and date of the sale.
Includes two Key columns, a large number of duplicate columns for the names of days of the week and months in three languages: English, French and Spanish. All the data types are correct.

In [10]:
date = spark.read.option('header', 'true').csv('Tables_WWI/DimDate.csv', inferSchema=True)
date.printSchema()

root
 |-- DateKey: integer (nullable = true)
 |-- FullDateAlternateKey: timestamp (nullable = true)
 |-- DayNumberOfWeek: integer (nullable = true)
 |-- EnglishDayNameOfWeek: string (nullable = true)
 |-- SpanishDayNameOfWeek: string (nullable = true)
 |-- FrenchDayNameOfWeek: string (nullable = true)
 |-- DayNumberOfMonth: integer (nullable = true)
 |-- DayNumberOfYear: integer (nullable = true)
 |-- WeekNumberOfYear: integer (nullable = true)
 |-- EnglishMonthName: string (nullable = true)
 |-- SpanishMonthName: string (nullable = true)
 |-- FrenchMonthName: string (nullable = true)
 |-- MonthNumberOfYear: integer (nullable = true)
 |-- CalendarQuarter: integer (nullable = true)
 |-- CalendarYear: integer (nullable = true)
 |-- CalendarSemester: integer (nullable = true)
 |-- FiscalQuarter: integer (nullable = true)
 |-- FiscalYear: integer (nullable = true)
 |-- FiscalSemester: integer (nullable = true)



The total number of days recorded: 4017

In [11]:
rows = date.count()
print(f"date DataFrame Rows count : {rows}")

date DataFrame Rows count : 4017


Extract the required data from the Date table using spark.sql:

In [12]:
date.createOrReplaceTempView('Date')
date_sql = spark.sql('SELECT DateKey, FullDateAlternateKey, DayNumberOfYear, EnglishMonthName, CalendarYear FROM date')
date_sql.show(5)

+--------+--------------------+---------------+----------------+------------+
| DateKey|FullDateAlternateKey|DayNumberOfYear|EnglishMonthName|CalendarYear|
+--------+--------------------+---------------+----------------+------------+
|20050101| 2005-01-01 00:00:00|              1|         January|        2005|
|20050102| 2005-01-02 00:00:00|              2|         January|        2005|
|20050103| 2005-01-03 00:00:00|              3|         January|        2005|
|20050104| 2005-01-04 00:00:00|              4|         January|        2005|
|20050105| 2005-01-05 00:00:00|              5|         January|        2005|
+--------+--------------------+---------------+----------------+------------+
only showing top 5 rows



Below we can see that the data record is from 2005-01-01 to 2015-12-31:

In [13]:
date_sql.select(min(date_sql.FullDateAlternateKey).alias("FirstDate"), 
          max(date_sql.FullDateAlternateKey).alias("LastDate")
    ).show()

+-------------------+-------------------+
|          FirstDate|           LastDate|
+-------------------+-------------------+
|2005-01-01 00:00:00|2015-12-31 00:00:00|
+-------------------+-------------------+



SALES TERRITORY TABLE

Also a small table containing information on the places where products are sold. It has two keys and contains 10 different countries.
All data types are correct.

In [14]:
sales_territory = spark.read.option('header', 'true').csv('Tables_WWI/DimSalesTerritory.csv', inferSchema=True)
sales_territory.printSchema()

root
 |-- SalesTerritoryKey: integer (nullable = true)
 |-- SalesTerritoryAlternateKey: integer (nullable = true)
 |-- SalesTerritoryRegion: string (nullable = true)
 |-- SalesTerritoryCountry: string (nullable = true)
 |-- SalesTerritoryGroup: string (nullable = true)



In [16]:
rows = sales_territory.count()
print(f"sales_territory DataFrame Rows count : {rows}")

sales_territory DataFrame Rows count : 11


In [17]:
sales_territory.createOrReplaceTempView('SalesTerritory')
sales_territory_sql = spark.sql('SELECT SalesTerritoryKey, SalesTerritoryCountry AS Country, SalesTerritoryRegion AS Region FROM SalesTerritory')
sales_territory_sql.show()

+-----------------+--------------+--------------+
|SalesTerritoryKey|       Country|        Region|
+-----------------+--------------+--------------+
|                1|           USA|     Northwest|
|                2|           USA|     Northeast|
|                3|           USA|       Central|
|                4|           USA|     Southwest|
|                5|           USA|     Southeast|
|                6|        Canada|        Canada|
|                7|        France|        France|
|                8|       Germany|       Germany|
|                9|     Australia|     Australia|
|               10|United Kingdom|United Kingdom|
|               11|            NA|            NA|
+-----------------+--------------+--------------+



PRODUCT TABLE

Describes the type, class, style and other parameters of the product.
Includes three Key columns, a large number of duplicate columns for ProductName in three languages: English, French and Spanish. Description dublicates in English, French, Chinese, Arabic, Hebrew, Thai, German, Japanese and Turkish.
All the data types are correct.

In [3]:
product = spark.read.option('header', 'true').csv('Tables_WWI/DimProduct.csv', inferSchema=True)
product.printSchema()

root
 |-- ProductKey: integer (nullable = true)
 |-- ProductAlternateKey: string (nullable = true)
 |-- ProductSubcategoryKey: string (nullable = true)
 |-- WeightUnitMeasureCode: string (nullable = true)
 |-- SizeUnitMeasureCode: string (nullable = true)
 |-- EnglishProductName: string (nullable = true)
 |-- SpanishProductName: string (nullable = true)
 |-- FrenchProductName: string (nullable = true)
 |-- StandardCost: string (nullable = true)
 |-- FinishedGoodsFlag: integer (nullable = true)
 |-- Color: string (nullable = true)
 |-- SafetyStockLevel: integer (nullable = true)
 |-- ReorderPoint: integer (nullable = true)
 |-- ListPrice: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- SizeRange: string (nullable = true)
 |-- Weight: string (nullable = true)
 |-- DaysToManufacture: integer (nullable = true)
 |-- ProductLine: string (nullable = true)
 |-- DealerPrice: string (nullable = true)
 |-- Class: string (nullable = true)
 |-- Style: string (nullable = true)
 |--

In [20]:
rows = product.count()
print(f"product DataFrame Rows count : {rows}")

product DataFrame Rows count : 606


Extract the required data from the Product table using spark.sql:

In [4]:
product.createOrReplaceTempView('Product')
product_sql = spark.sql('SELECT ProductKey, ProductSubcategoryKey, EnglishProductName AS ProductName FROM Product')
product_sql.show(5)

+----------+---------------------+--------------------+
|ProductKey|ProductSubcategoryKey|         ProductName|
+----------+---------------------+--------------------+
|         1|                 NULL|     Adjustable Race|
|         2|                 NULL|        Bearing Ball|
|         3|                 NULL|     BB Ball Bearing|
|         4|                 NULL|Headset Ball Bear...|
|         5|                 NULL|               Blade|
+----------+---------------------+--------------------+
only showing top 5 rows



ProductSubcategoryKey values need to be converted from string to integer. This makes it possible to create a relation between the product and product_subcategory tables by the ProductSubcategoryKey. 

In [5]:
product_sql = product_sql.withColumn("ProductSubcategoryKey", product_sql["ProductSubcategoryKey"].cast(IntegerType()))

In [6]:
product_sql.printSchema()

root
 |-- ProductKey: integer (nullable = true)
 |-- ProductSubcategoryKey: integer (nullable = true)
 |-- ProductName: string (nullable = true)



Checking for NULL values of ProductSubcategoryKey column.

In [23]:
spark.sql('SELECT COUNT(ProductKey) FROM Product WHERE ProductSubcategoryKey IS NULL').show()


+-----------------+
|count(ProductKey)|
+-----------------+
|              209|
+-----------------+



PRODUCT SUBCATEGORY TABLE

A small table containing information about a sub-category of a product. Has two Keys and contains 37 different types of sub-categories.
All the data types are correct.

In [24]:
product_subcategory = spark.read.option('header', 'true').csv('Tables_WWI/DimProductSubcategory.csv', inferSchema=True)
product_subcategory.printSchema()

root
 |-- ProductSubcategoryKey: integer (nullable = true)
 |-- ProductSubcategoryAlternateKey: integer (nullable = true)
 |-- EnglishProductSubcategoryName: string (nullable = true)
 |-- SpanishProductSubcategoryName: string (nullable = true)
 |-- FrenchProductSubcategoryName: string (nullable = true)
 |-- ProductCategoryKey: integer (nullable = true)



In [26]:
rows = product_subcategory.count()
print(f"product_subcategory DataFrame Rows count : {rows}")

product_subcategory DataFrame Rows count : 37


In [27]:
product_subcategory.createOrReplaceTempView('ProductSubcategory')
product_subcategory_sql = spark.sql('SELECT ProductSubcategoryKey, ProductCategoryKey, EnglishProductSubcategoryName AS SubCategory FROM ProductSubcategory')
product_subcategory_sql.show(5)

+---------------------+------------------+---------------+
|ProductSubcategoryKey|ProductCategoryKey|    SubCategory|
+---------------------+------------------+---------------+
|                    1|                 1| Mountain Bikes|
|                    2|                 1|     Road Bikes|
|                    3|                 1|  Touring Bikes|
|                    4|                 2|     Handlebars|
|                    5|                 2|Bottom Brackets|
+---------------------+------------------+---------------+
only showing top 5 rows



Checking for unique SubCategory values:

In [29]:
dist = product_subcategory_sql.select(countDistinct("SubCategory"))
dist.show()

+---------------------------+
|count(DISTINCT SubCategory)|
+---------------------------+
|                         37|
+---------------------------+



PRODUCT CATEGORY TABLE

Similar to product subcategory table. Contains important information about the main categories of products. It has two keys and all data types are correct.

In [30]:
product_category = spark.read.option('header', 'true').csv('Tables_WWI/ProductCategory.csv', inferSchema=True)
product_category.printSchema()

root
 |-- ProductCategoryKey: integer (nullable = true)
 |-- ProductCategoryAlternateKey: integer (nullable = true)
 |-- EnglishProductCategoryName: string (nullable = true)
 |-- SpanishProductCategoryName: string (nullable = true)
 |-- FrenchProductCategoryName: string (nullable = true)



Below we see that there are 4 unique types of product categories:

In [31]:
product_category.createOrReplaceTempView('ProductCategory')
product_category_sql = spark.sql('SELECT ProductCategoryKey, EnglishProductCategoryName AS Category FROM ProductCategory')
product_category_sql.show()

+------------------+-----------+
|ProductCategoryKey|   Category|
+------------------+-----------+
|                 1|      Bikes|
|                 2| Components|
|                 3|   Clothing|
|                 4|Accessories|
+------------------+-----------+



JOINING TABLES

Firstly, it was decided to combine all the tables relating to product features, such as product_subcategory_sql, product_category_sql and finally product_sql. 

In [32]:
category_table = product_subcategory_sql.join(product_category_sql, ['ProductCategoryKey'], 'inner')
category_table.show(3)

+------------------+---------------------+--------------+--------+
|ProductCategoryKey|ProductSubcategoryKey|   SubCategory|Category|
+------------------+---------------------+--------------+--------+
|                 1|                    1|Mountain Bikes|   Bikes|
|                 1|                    2|    Road Bikes|   Bikes|
|                 1|                    3| Touring Bikes|   Bikes|
+------------------+---------------------+--------------+--------+
only showing top 3 rows



In [33]:
product_table = product_sql.join(category_table, ['ProductSubcategoryKey'], 'inner')
product_table.show(3)

+---------------------+----------+--------------------+------------------+--------------+--------+
|ProductSubcategoryKey|ProductKey|         ProductName|ProductCategoryKey|   SubCategory|Category|
+---------------------+----------+--------------------+------------------+--------------+--------+
|                    1|       600|Mountain-500 Blac...|                 1|Mountain Bikes|   Bikes|
|                    1|       599|Mountain-500 Blac...|                 1|Mountain Bikes|   Bikes|
|                    1|       598|Mountain-500 Blac...|                 1|Mountain Bikes|   Bikes|
+---------------------+----------+--------------------+------------------+--------------+--------+
only showing top 3 rows



Afterwards profit_sale_sql, date_sql, sales_territory_sql and product_table were joined.

In [86]:
analytical_table = profit_sale_sql.join(date_sql).where(profit_sale_sql['OrderDateKey'] == date_sql['DateKey']).join(sales_territory_sql, ['SalesTerritoryKey']).join(product_table, ['ProductKey'])
analytical_table.show(3)

+----------+-----------------+------------+---------+----------------+-------+------+------------+--------------+------------------+--------+--------------------+---------------+----------------+------------+---------+---------+---------------------+--------------------+------------------+-----------+--------+
|ProductKey|SalesTerritoryKey|OrderDateKey|UnitsSold|TotalProductCost|Revenue| Taxes|DeliveryCost|RevenueRevenue|            Profit| DateKey|FullDateAlternateKey|DayNumberOfYear|EnglishMonthName|CalendarYear|  Country|   Region|ProductSubcategoryKey|         ProductName|ProductCategoryKey|SubCategory|Category|
+----------+-----------------+------------+---------+----------------+-------+------+------------+--------------+------------------+--------+--------------------+---------------+----------------+------------+---------+---------+---------------------+--------------------+------------------+-----------+--------+
|       463|                4|    20130703|        1|          9

In [87]:
analytical_table.printSchema()

root
 |-- ProductKey: integer (nullable = true)
 |-- SalesTerritoryKey: integer (nullable = true)
 |-- OrderDateKey: integer (nullable = true)
 |-- UnitsSold: integer (nullable = true)
 |-- TotalProductCost: double (nullable = true)
 |-- Revenue: double (nullable = true)
 |-- Taxes: double (nullable = true)
 |-- DeliveryCost: double (nullable = true)
 |-- RevenueRevenue: integer (nullable = true)
 |-- Profit: double (nullable = true)
 |-- DateKey: integer (nullable = true)
 |-- FullDateAlternateKey: timestamp (nullable = true)
 |-- DayNumberOfYear: integer (nullable = true)
 |-- EnglishMonthName: string (nullable = true)
 |-- CalendarYear: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- ProductSubcategoryKey: integer (nullable = true)
 |-- ProductName: string (nullable = true)
 |-- ProductCategoryKey: integer (nullable = true)
 |-- SubCategory: string (nullable = true)
 |-- Category: string (nullable = true)



The new table is an analytical base table (ABT) that is used for data analysis and visualisation, it has the same number of rows as in main table fact_internet_sale:

In [43]:
rows = analytical_table.count()
print(f"analytical_table DataFrame Rows count : {rows}")

analytical_table DataFrame Rows count : 60398


SUM OF UnitsSold, Revenue AND Profit:

In [97]:
analytical_table.createOrReplaceTempView('AnalyticalTable')
spark.sql('SELECT SUM(UnitsSold), SUM(Revenue), ROUND(SUM(Profit), 2) FROM AnalyticalTable ORDER BY SUM(UnitsSold) DESC, SUM(Revenue) DESC, SUM(Profit) DESC').show()

+--------------+-------------------+---------------------+
|sum(UnitsSold)|       sum(Revenue)|round(sum(Profit), 2)|
+--------------+-------------------+---------------------+
|         60398|2.935867722065187E7|           9732189.41|
+--------------+-------------------+---------------------+



UnitsSold AND Profit BY COUNTRY:

In [72]:
spark.sql('SELECT Country, COUNT(UnitsSold), ROUND(SUM(Profit), 2) FROM AnalyticalTable GROUP BY Country ORDER BY COUNT(UnitsSold) DESC, SUM(Profit) DESC').show()

+--------------+----------------+---------------------+
|       Country|count(UnitsSold)|round(sum(Profit), 2)|
+--------------+----------------+---------------------+
|           USA|           21344|           3149797.63|
|     Australia|           13345|           2960975.01|
|        Canada|            7620|            671693.91|
|United Kingdom|            6906|            1119153.8|
|       Germany|            5625|            955825.77|
|        France|            5558|             874743.3|
+--------------+----------------+---------------------+



In [ ]:
PROFIT BY Category:

In [71]:
spark.sql('SELECT Category, ROUND(SUM(Profit), 2) FROM AnalyticalTable GROUP BY Category ORDER BY SUM(Profit) DESC').show()

+-----------+---------------------+
|   Category|round(sum(Profit), 2)|
+-----------+---------------------+
|      Bikes|           9240344.88|
|Accessories|            382613.77|
|   Clothing|            109230.77|
+-----------+---------------------+



In [ ]:
PROFIT BY SubCategory:

In [69]:
spark.sql('SELECT SubCategory, ROUND(SUM(Profit), 2) FROM AnalyticalTable GROUP BY SubCategory ORDER BY SUM(Profit) DESC').show()

+-----------------+---------------------+
|      SubCategory|round(sum(Profit), 2)|
+-----------------+---------------------+
|       Road Bikes|           4375652.96|
|   Mountain Bikes|            3717403.3|
|    Touring Bikes|           1147288.61|
|  Tires and Tubes|            134058.41|
|          Helmets|            123032.98|
|           Shorts|             38940.58|
|Bottles and Cages|             31011.49|
|          Jerseys|              25942.6|
|          Fenders|             25454.33|
|  Hydration Packs|             22007.96|
|      Bike Stands|             21616.69|
|       Bike Racks|             21490.56|
|            Vests|              19485.1|
|           Gloves|             19121.24|
|         Cleaners|              3941.36|
|             Caps|              2953.22|
|            Socks|              2788.03|
+-----------------+---------------------+



SUM OF TotalProductCost, Revenue, Profit BY CalendarYear:

In [65]:
spark.sql('SELECT CalendarYear, SUM(TotalProductCost), SUM(Revenue), SUM(Profit) FROM AnalyticalTable GROUP BY CalendarYear ORDER BY CalendarYear DESC').show()

+------------+---------------------+------------------+------------------+
|CalendarYear|sum(TotalProductCost)|      sum(Revenue)|       sum(Profit)|
+------------+---------------------+------------------+------------------+
|        2014|    5721205.239699934|  9770899.74000497|3268022.5210993364|
|        2013|     5718327.17489979|  9791060.29770275|3289448.2729995847|
|        2012|    3883493.374599957|  6530343.52639983| 2124422.648200007|
|        2011|   1954767.7864999673|3266373.6566000194|1050295.9726000114|
+------------+---------------------+------------------+------------------+



SAVE CSV FILE:

In [25]:
analytic_table.coalesce(1).write.csv('Tables_WWI/AnalyticTable.csv', mode='overwrite', header=True)

CONCLUSIONS:

- The Wide World Importers Dataset takes 20.4 MB and consist of 6 tabs. 
- It was discovered, that the main table is FactSales.csv, that stores facts related to a business operations, it is located at the center of a schema and is supplemented by the dimension tables: DimCustomer.csv, DimEmployee.xlxs, DimStockItem.csv, DimDate.csv and DimCity.csv.
- Data architecture has been studied: the dataset structure is a Snowflake - a multi-dimensional data model that is similar to a star schema, but where dimension tables are broken down into subdimensions? as it is in all the tables relating to product features (product_subcategory_sql, product_category_sql and product_sql).
- Each WWI table has between 2-8 Keys, which ensures linking between the tables, appropriate use and storage of the data.
- Was discovered that 60 000 units were sold, total sum of revenue is 29.36M dollars and total profit is 9.73M.
- The biggest profit is in 2014 year - 3268022.5 despite the fact that there is data only from January to start of July. When it comes to category the biggest profit is from Bikes - 95%.
- The data visualisation is made with PowerBI and presented as a pdf file.